import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load
df_cc = pd.read_csv('../data/raw/creditcard.csv')
print(df_cc.shape)
print(df_cc.head())

In [ ]:
# Log transform Amount.
df_cc['log_amount'] = np.log(df_cc['Amount'] + 1)

# Class distribution
print(df_cc['Class'].value_counts(normalize=True))  # 0.17% fraud

fig, axes = plt.subplots(2, 2, figsize=(12,8))
sns.histplot(df_cc['Time'], kde=True, ax=axes[0,0])
axes[0,0].set_title('Time')

sns.histplot(df_cc['Amount'], kde=True, ax=axes[0,1])
axes[0,1].set_title('Amount (Skewed)')

sns.histplot(df_cc['log_amount'], kde=True, ax=axes[1,0])
axes[1,0].set_title('Log Amount')

# Sample V1-V28 (too many; show V1)
sns.histplot(df_cc['V1'], kde=True, ax=axes[1,1])
axes[1,1].set_title('V1 (PCA Feature)')
plt.tight_layout()
plt.show()

In [ ]:
# Fraud by Amount
sns.boxplot(data=df_cc, x='Class', y='Amount')
plt.title('Amount by Class')
plt.show()

# Correlations with Class (select top)
corr_cols = [col for col in df_cc.columns if col.startswith('V')]
corr_df = df_cc[corr_cols + ['Class']].corr()['Class'].abs().sort_values(ascending=False)[1:11]
print(corr_df)

sns.heatmap(df_cc[['V14', 'V17', 'V12', 'V10', 'Class']].corr(), annot=True, cmap='coolwarm')
plt.title('Top Correlated Features with Class')
plt.show()

plt.pie(df_cc['Class'].value_counts(), labels=['Non-Fraud', 'Fraud'], autopct='%1.2f%%')
plt.title('Class Imbalance (0.17% Fraud)')
plt.show()

In [ ]:
df_cc.to_parquet('../data/processed/creditcard_processed.parquet')

# EDA: Credit Card Fraud Dataset
Analyzing anonymized bank transactions for fraud patterns. Focus: Imbalance, Amount skew, PCA feature correlations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")

# Load data
df_cc = pd.read_csv('../data/raw/creditcard.csv')
print(f"Dataset shape: {df_cc.shape}")
print(df_cc.head())
print("\nTarget distribution:")
print(df_cc['Class'].value_counts(normalize=True))

In [ ]:
# No misses/dups, but transform skew
df_cc['log_amount'] = np.log1p(df_cc['Amount'])  # log(1 + x) for zeros

# Time to hours (cyclic later)
df_cc['time_hours'] = df_cc['Time'] / 3600 % 24  # 0-24 hours

print("Transformed columns added: log_amount, time_hours")
print(f"Amount skew before: {df_cc['Amount'].skew():.2f}, after log: {df_cc['log_amount'].skew():.2f}")
print("No missing values:", df_cc.isnull().sum().sum() == 0)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Univariate Distributions (Credit Card Data)', fontsize=16)

# Time
sns.histplot(df_cc['Time'], bins=50, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Time (Seconds from Start)')

# Amount (raw)
sns.histplot(df_cc['Amount'], bins=50, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Transaction Amount (Skewed)')

# Log Amount
sns.histplot(df_cc['log_amount'], bins=50, kde=True, ax=axes[0, 2])
axes[0, 2].set_title('Log(Amount + 1)')

# Sample PCA features (V1, V3 for variety)
sns.histplot(df_cc['V1'], bins=50, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('V1 (PCA Feature)')

sns.histplot(df_cc['V3'], bins=50, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('V3 (PCA Feature)')

# Class (pie for imbalance)
class_pie = df_cc['Class'].value_counts()
axes[1, 2].pie(class_pie.values, labels=['Non-Fraud', 'Fraud'], autopct='%1.1f%%', colors=['lightblue', 'salmon'])
axes[1, 2].set_title('Class Imbalance')

plt.tight_layout()
plt.show()